In [1]:
from lambeq import BobcatParser, TreeReader, TreeReaderMode, spiders_reader, cups_reader, stairs_reader
from lambeq import TensorAnsatz, SpiderAnsatz, MPSAnsatz, AtomicType
from discopy import Dim
from classic_pipeline import *
from utilities import *

/home/adriano22_/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#define atomic-types

N = AtomicType.NOUN
S = AtomicType.SENTENCE
C = AtomicType.CONJUNCTION
P = AtomicType.PUNCTUATION
NP = AtomicType.NOUN_PHRASE
PP = AtomicType.PREPOSITIONAL_PHRASE

In [3]:
#parser declaration

bobcat_parser = BobcatParser(verbose = "progress")
spider_parser = spiders_reader
cups_parser = cups_reader
stairs_parser = stairs_reader
tree_parser = TreeReader(mode=TreeReaderMode.RULE_ONLY)

In [4]:
#ansatze declaration

tensor_ansatz = TensorAnsatz({N: Dim(2), S: Dim(2), C: Dim(2), P: Dim(2), NP: Dim(2), PP: Dim(2)})

In [ ]:
#data-extraction for classic pipeline (linux)

pip = ClassicPipeline(bobcat_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
#train_labels, train_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/GPS_edited.csv", "n")
#test_labels, test_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/CPN_edited.csv", "n")
#eval_labels, eval_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/ePurse_edited.csv", "n")

In [ ]:
#data-extraction for classic pipeline (win11)

pip = ClassicPipeline(bobcat_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
train_labels, train_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\GPS_edited.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\CPN_edited.csv", "n")
eval_labels, eval_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\ePurse_edited.csv", "n")

In [9]:
#save_data("train_data.txt", train_labels, train_circuits)
#save_data("test_data.txt", test_labels, test_circuits)
#save_data("eval_data.txt", eval_labels, eval_circuits)

train_labels, train_circuits = load_data("train_data.txt")
test_labels, test_circuits = load_data("test_data.txt")
eval_labels, eval_circuits = load_data("eval_data.txt")

In [10]:
#train-set (GPS) faulty entries: 127:128 128:129 129:130 136:137 137:138 138:139 147:148 150:151 151:152 155:156 156:157
#test-set (CPN) faulty entries: None
#eval-set (ePurse) faulty entries: 22:23 34:46


rtrain_circuits = train_circuits[0:127] + train_circuits[130:136] + train_circuits[139:147] + train_circuits[148:150] + train_circuits[152:155] + train_circuits[157:]
reval_circuits = eval_circuits[0:22] + eval_circuits[23:34] + eval_circuits[46:]


In [ ]:
#training block for classical pipeline

train_set, test_set, eval_set = pip.create_dataset(train_labels[0:157], rtrain_circuits), pip.create_dataset(test_labels, test_circuits), pip.create_dataset(eval_labels[0:111], reval_circuits)
pip.create_trainer(rtrain_circuits, test_circuits, reval_circuits)
pip.train_model(train_set, eval_set)
pip.plot() 

In [22]:
train_tensors = []
test_tensors = []

for circuit in train_circuits:
    symbols = set(circuit.free_symbols)
    symbol2index = {symbol: i for i, symbol in enumerate(symbols)}

    N = len(symbols)
    matrix = np.zeros((N, N), dtype=bool)
    for i, symbol_i in enumerate(symbols):
        for j, symbol_j in enumerate(symbols):
            if any(box.dom == symbol_i and box.cod == symbol_j for box in circuit.boxes):
                matrix[i, j] = True

    tensor = torch.tensor(np.stack([matrix, matrix], axis=-1), dtype=torch.bool)
    train_tensors.append(tensor)

for circuit in test_circuits:
    symbols = set(circuit.free_symbols)
    symbol2index = {symbol: i for i, symbol in enumerate(symbols)}

    N = len(symbols)
    matrix = np.zeros((N, N), dtype=bool)
    for i, symbol_i in enumerate(symbols):
        for j, symbol_j in enumerate(symbols):
            if any(box.dom == symbol_i and box.cod == symbol_j for box in circuit.boxes):
                matrix[i, j] = True

    tensor = torch.tensor(np.stack([matrix, matrix], axis=-1), dtype=torch.bool)
    test_tensors.append(tensor)

In [ ]:
from lambeq import BobcatParser, AtomicType
from lambeq import TensorAnsatz
from discopy import Box, Ty, Dim, Word
# Define atomic types
N = AtomicType.NOUN
S = AtomicType.SENTENCE

sentence = 'John walks in the park'

# Get a string diagram
parser = BobcatParser(verbose='text')
tensor_ansatz = TensorAnsatz({N: Dim(2), S: Dim(2)})
diagram = parser.sentence2diagram(sentence)
tensor_diagram = tensor_ansatz(diagram)


pad_diagrams = [tensor_ansatz(parser.sentence2diagram("parola")) for i in range(1, 20)]

new_diagram = tensor_diagram
for pad in pad_diagrams:
    new_diagram = pad @ new_diagram

print(new_diagram)
print(len(new_diagram))
print(len(tensor_diagram))
